# Customer Lifetime Value (CLV) Prediction

In [ ]:
# Customer Lifetime Value (CLV) is the total revenue a business can expect from a customer over the entire duration of their relationship. 
# It helps businesses understand how valuable a customer is beyond just a single purchase.

In [ ]:
# this is more of prediction based analysis rather than descriptive or dignostic analysis

In [ ]:
# importing requred pakages

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# poitning respective datasets

In [3]:
path_orders = r"C:\Users\Vinayak\Data Analyst - Raw Data (1)\formatted_datasets\orders_formatted.csv"
path_customers = r"C:\Users\Vinayak\Data Analyst - Raw Data (1)\formatted_datasets\customers_formatted.csv"
path_products = r"C:\Users\Vinayak\Data Analyst - Raw Data (1)\formatted_datasets\products_std_fotmatted.csv"
path_order_items = r"C:\Users\Vinayak\Data Analyst - Raw Data (1)\Senior Data Analyst - Raw Data\order_items.csv"
path_cust_rating = r"C:\Users\Vinayak\Data Analyst - Raw Data (1)\formatted_datasets\customer_reviews_formatted.csv"

In [ ]:
# creating dataframes

In [5]:
df_orders = pd.read_csv(path_orders)
df_customers = pd.read_csv(path_customers)
df_products = pd.read_csv(path_products)
df_order_items = pd.read_csv(path_order_items)
df_cust_rating = pd.read_csv(path_cust_rating)

In [ ]:
# staderdising date values

In [9]:
df_orders['order_date']=pd.to_datetime(df_orders['order_date'])

In [11]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      30 non-null     int64         
 1   customer_id   30 non-null     int64         
 2   order_date    30 non-null     datetime64[ns]
 3   total_amount  30 non-null     int64         
 4   status        30 non-null     object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.3+ KB


# Average Order Value (AOV)

In [ ]:
# before building model we will need few calcuated fields
# Average Order Value (AOV) = Total Revenue / Total Orders
# Purchase Frequency (PF) = Total Orders / Total Customers
# Recency (R) = Days since last purchase

In [ ]:
# out of them AOV and puchase frequnecy is calculated as below

In [25]:
df_pre=df_orders.groupby('customer_id').agg({
        'total_amount':'sum',
        'order_id':'count'    
}).reset_index()
df_pre['AOV'] = df_pre['total_amount'] / df_pre['order_id']
df_pre.rename(columns={
    'customer_id':'customer id',
    'total_amount':'order value',
    'order_id':'order frequency',
    'AOV':'avg order val'
})

,customer id,order value,order frequency,avg order val
0,101,1400,4,350.000000
1,102,1050,3,350.000000
2,103,1350,3,450.000000
3,104,1650,3,550.000000
4,105,1070,3,356.666667
5,106,870,3,290.000000
6,107,1350,3,450.000000
7,108,650,2,325.000000
8,109,650,2,325.000000
9,110,800,2,400.000000


# Recency (R)

In [ ]:
# recency is calcualetd based on last order places by each customer

In [27]:
df_date=df_orders['order_date'].max()
df_date

Timestamp('2023-12-20 00:00:00')

In [ ]:
# by taking most recent order date from dataset and comaring other dates against it

In [29]:
df_recency = df_orders.groupby('customer_id').agg({
    'order_date':'max'
}).reset_index()
df_recency['recency'] = (df_date - df_recency['order_date']).dt.days
df_recency

,customer_id,order_date,recency
0,101,2023-10-05,76
1,102,2023-11-05,45
2,103,2023-11-12,38
3,104,2023-12-01,19
4,105,2023-12-10,10
5,106,2023-12-15,5
6,107,2023-12-20,0
7,108,2023-09-05,106
8,109,2023-09-10,101
9,110,2023-09-15,96


In [ ]:
# merging AOV datset with recency dataset to get final dataset as below

In [31]:
df_final=df_pre.merge(df_recency,on='customer_id')
df_final

,customer_id,total_amount,order_id,AOV,order_date,recency
0,101,1400,4,350.000000,2023-10-05,76
1,102,1050,3,350.000000,2023-11-05,45
2,103,1350,3,450.000000,2023-11-12,38
3,104,1650,3,550.000000,2023-12-01,19
4,105,1070,3,356.666667,2023-12-10,10
5,106,870,3,290.000000,2023-12-15,5
6,107,1350,3,450.000000,2023-12-20,0
7,108,650,2,325.000000,2023-09-05,106
8,109,650,2,325.000000,2023-09-10,101
9,110,800,2,400.000000,2023-09-15,96


In [ ]:
# Calculating CLV as 
# CLV = AOV * frequency

In [33]:
df_aov_input=df_final[['customer_id','AOV','order_id','recency']]
df_aov_input=df_aov_input.rename(columns={
    'customer_id':'customer id',
    'order_id':'order freq',
})
df_aov_input['clv']=df_aov_input['AOV'] * df_aov_input['order freq']
df_aov_input

,customer id,AOV,order freq,recency,clv
0,101,350.000000,4,76,1400.0
1,102,350.000000,3,45,1050.0
2,103,450.000000,3,38,1350.0
3,104,550.000000,3,19,1650.0
4,105,356.666667,3,10,1070.0
5,106,290.000000,3,5,870.0
6,107,450.000000,3,0,1350.0
7,108,325.000000,2,106,650.0
8,109,325.000000,2,101,650.0
9,110,400.000000,2,96,800.0


In [ ]:
# exporting a datafrem into csv file

In [35]:
df_aov_input.to_csv(r'C:\Users\Vinayak\Data Analyst - Raw Data (1)\aggrigated_datasets\CLV_dataset.csv')

# Predicitng CLV 

# Import requred Lib

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Load the Data

In [39]:
df_aov_input

,customer id,AOV,order freq,recency,clv
0,101,350.000000,4,76,1400.0
1,102,350.000000,3,45,1050.0
2,103,450.000000,3,38,1350.0
3,104,550.000000,3,19,1650.0
4,105,356.666667,3,10,1070.0
5,106,290.000000,3,5,870.0
6,107,450.000000,3,0,1350.0
7,108,325.000000,2,106,650.0
8,109,325.000000,2,101,650.0
9,110,400.000000,2,96,800.0


In [ ]:
# check datatypes

In [43]:
df_aov_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer id  11 non-null     int64  
 1   AOV          11 non-null     float64
 2   order freq   11 non-null     int64  
 3   recency      11 non-null     int64  
 4   clv          11 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 572.0 bytes


In [ ]:
# renaming columns that suits model coding

In [45]:
df_aov_input.columns= ['customer_id', 'aov', 'order_freq', 'recency', 'clv']
df_aov_input

,customer_id,aov,order_freq,recency,clv
0,101,350.000000,4,76,1400.0
1,102,350.000000,3,45,1050.0
2,103,450.000000,3,38,1350.0
3,104,550.000000,3,19,1650.0
4,105,356.666667,3,10,1070.0
5,106,290.000000,3,5,870.0
6,107,450.000000,3,0,1350.0
7,108,325.000000,2,106,650.0
8,109,325.000000,2,101,650.0
9,110,400.000000,2,96,800.0


In [ ]:
# making sure data is numeric type only

In [47]:
df_aov_input['clv'] = pd.to_numeric(df_aov_input['clv'], errors='coerce')

In [51]:
df_aov_input.dtypes

customer_id      int64
aov            float64
order_freq       int64
recency          int64
clv            float64
dtype: object

In [ ]:
# checking nulls

In [61]:
df_aov_input.isna().sum()

customer_id    0
aov            0
order_freq     0
recency        0
clv            0
dtype: int64

In [ ]:
# checking infinte values

In [63]:
np.isinf(df_aov_input)

,customer_id,aov,order_freq,recency,clv
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
9,False,False,False,False,False


In [ ]:
# deviding and assiging x and y parts

In [ ]:
# defining feature and target
# feature = x = 'AOV', 'order_freq', 'recency'
# target = y = clv

In [ ]:
# reassuring the dataframe has only numeric datatype

In [67]:
X = df_aov_input[['aov','order_freq','recency']].astype(float)
y = df_aov_input['clv'].astype(float)

In [71]:
# initialising model

In [73]:
model= LinearRegression()
model.fit(X,y)
Y_predict = model.predict(X)


In [ ]:
# deviding test and training part

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [79]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# making predictions 

In [81]:
y_pred = model.predict(X_test)

In [ ]:
# applying evaluatio methods to check accuracy of model

In [83]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [85]:
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

RMSE: 54.49
R² Score: 0.97


In [ ]:
# RMSE: 54.49 – this depends upon range of CLV.
# R² Score: 0.97 - 97% of the variability in CLV is explained by your features (aov, order_freq, recency). 
# The model fits the data well